In [4]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


def load_data():
    mnist = datasets.fetch_openml("mnist_784", version=1, as_frame=False)
    X, y = mnist.data, mnist.target
    y = y.astype(np.int32)  # Convert target to integers
    return X, y


def split_data(X, y, test_size=0.2, random_state=42):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)


def avg_pooling(image, pool_size):
    pooled_height = image.shape[0] // pool_size[0]
    pooled_width = image.shape[1] // pool_size[1]
    pooled_image = np.zeros((pooled_height, pooled_width))

    for i in range(pooled_height):
        for j in range(pooled_width):
            start_i, end_i = i * pool_size[0], (i + 1) * pool_size[0]
            start_j, end_j = j * pool_size[1], (j + 1) * pool_size[1]
            pooled_image[i, j] = np.mean(image[start_i:end_i, start_j:end_j])

    return pooled_image


def preprocess_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled

In [5]:
import tqdm

# Load and split the data
X, y = load_data()

pool_size = 3
X_pooled = np.zeros((X.shape[0], (28 // pool_size) * (28 // pool_size)))
for i in tqdm.tqdm(range(X.shape[0])):
    image = X[i].reshape(28, 28)
    pooled_image = avg_pooling(image, (pool_size, pool_size))
    X_pooled[i] = pooled_image.flatten()

X_train, X_test, y_train, y_test = split_data(X_pooled, y)

# Preprocess the data
X_train_scaled, X_test_scaled = preprocess_data(X_train, X_test)

c:\Users\Parisa\anaconda3\envs\neural_bandits\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
100%|██████████| 70000/70000 [00:47<00:00, 1469.44it/s]


In [6]:
import sys
sys.path.append('D:\calvera\src')

In [7]:
import torch.nn as nn

class Network(nn.Module):
    def __init__(self, dim, hidden_size=100):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(dim, hidden_size)
        self.activate = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
    def forward(self, x):
        return self.fc2(self.activate(self.fc1(x)))

In [36]:
import torch

from neural_bandits.algorithms.neural_ucb_bandit import NeuralUCB
from neural_bandits.trainers.neural_ucb_trainer import NeuralUCBTrainer
from neural_bandits.utils.multiclass import MultiClassContextualiser

def train_neural_ucb_mnist(X_train_scaled, y_train, n_samples: int = 10000):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    n_features = X_train_scaled.shape[1]
    n_arms = 10

    network = Network(n_features * n_arms, hidden_size=100)
    
    # Initialize bandit
    bandit = NeuralUCB(
        network=network,
        n_arms=n_arms,
        n_features=n_features * n_arms,
        lambda_=0.00001,
        nu=0.00001,
    ).to(device)
    
    trainer = NeuralUCBTrainer(eta=0.01)
    mc_contextualiser = MultiClassContextualiser(n_arms)
    
    total_regret = []
    
    for t in tqdm.tqdm(range(n_samples)):
        x_tensor = torch.tensor(X_train_scaled[t], dtype=torch.float32).to(device).reshape(1, -1)
        contextualised_actions = mc_contextualiser.contextualise(x_tensor)
        # print("contextualised_actions.shape: ", contextualised_actions.shape)
        
        # Select action
        ucb_scores = bandit(contextualised_actions)
        chosen_arm = torch.argmax(ucb_scores, dim=1).item()
        
        # Get reward
        reward = 1.0 if y_train[t] == chosen_arm else 0.0
        reward = torch.tensor([reward], dtype=torch.float32, device=device).reshape(1, -1)
        total_regret.append(1 - reward.item())
        
        # Extract features for chosen arm
        contextualised_actions = contextualised_actions.squeeze(0)
        chosen_features = contextualised_actions[chosen_arm]
        # print("chosen_features.shape: ", chosen_features.shape)
        
        # Update bandit
        if t < 1000:
            bandit = trainer.update(bandit, reward, chosen_features)
        else:
            if t % 100 == 0:
                bandit = trainer.update(bandit, reward, chosen_features)

        if t % 100 == 0:
            print(f"Step {t}:")
            print(f"UCB Scores: {ucb_scores}")
            print(f"Chosen arm: {chosen_arm}")
            print(f"True label: {y_train[t]}")
            print(f"Average regret: {sum(total_regret)/(t+1)}")
    
    return bandit, total_regret

In [37]:
# Train the neural UCB bandit
bandit, regret = train_neural_ucb_mnist(X_train_scaled, y_train)

# Print results
print("Total regret:", sum(regret))
print("Average regret:", sum(regret) / len(regret))

  0%|          | 5/10000 [00:00<03:46, 44.04it/s]

Step 0:
UCB Scores: tensor([[0.0910, 0.1041, 0.1001, 0.1032, 0.1025, 0.1057, 0.0986, 0.1043, 0.1010,
         0.0893]])
Chosen arm: 5
True label: 5
Average regret: 0.0


  1%|          | 102/10000 [00:12<30:47,  5.36it/s]

Step 100:
UCB Scores: tensor([[0.1668, 0.0742, 0.0818, 0.0819, 0.1043, 0.1183, 0.1252, 0.0622, 0.0899,
         0.0953]])
Chosen arm: 0
True label: 0
Average regret: 0.6534653465346535


  2%|▏         | 202/10000 [00:41<37:57,  4.30it/s]  

...

Step 9800:
UCB Scores: tensor([[0.2200, 0.0985, 0.1159, 0.0733, 0.0870, 0.0606, 0.1491, 0.1092, 0.0542,
         0.0324]])
Chosen arm: 0
True label: 0
Average regret: 0.18181818181818182


 99%|█████████▉| 9934/10000 [09:31<00:00, 69.24it/s] 

Step 9900:
UCB Scores: tensor([[0.1010, 0.0494, 0.0772, 0.0859, 0.0878, 0.2174, 0.1413, 0.0910, 0.0894,
         0.0596]])
Chosen arm: 5
True label: 5
Average regret: 0.1812948187051813


100%|██████████| 10000/10000 [09:32<00:00, 17.48it/s]

Total regret: 1811.0
Average regret: 0.1811


In [38]:
import matplotlib.pyplot as plt

def plot_regret(regret):
    # Convert to numpy array if it's a list
    regret = np.array(regret)
    
    # Plot cumulative regret
    plt.figure(figsize=(10, 6))
    cumulative_regret = np.cumsum(regret)
    plt.plot(cumulative_regret, label='Cumulative Regret')
    plt.xlabel('Time Step')
    plt.ylabel('Cumulative Regret')
    plt.title('Neural UCB Cumulative Regret Over Time')
    plt.grid(True)
    plt.legend()
    plt.show()
    
    # Plot average regret
    plt.figure(figsize=(10, 6))
    steps = np.arange(1, len(cumulative_regret) + 1)  # create step array first
    average_regret = cumulative_regret / steps
    plt.plot(average_regret, label='Average Regret')
    plt.xlabel('Time Step')
    plt.ylabel('Average Regret')
    plt.title('Neural UCB Average Regret Over Time')
    plt.grid(True)
    plt.legend()
    plt.show()
# After training:
plot_regret(regret)